In [1]:
from lib2to3.pgen2.pgen import DFAState
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)


In [17]:
def clean(df, type):
    df = df.dropna(how='all')
    df = df.reset_index()
    if type == 'bat':
        df = df.drop(columns=['Unnamed: 8', 'Unnamed: 9'])
        df = df.rename(columns={"Unnamed: 1": "Dismissal"})
    elif type == 'bowl':
        list1 = []
        for i in range(len(df)):
            if len(df.iloc[i]['BOWLING'].split(' ')) > 5:
                list1.append(i)
        for i in list1:
            df = df.drop(i)
    df = df.drop(columns=['index'])
    
    s = df.style.set_table_styles([
                            {
                                "selector":"thead",
                                "props": [("background-color", "dodgerblue"), ("color", "white"),
                                          ("border", "3px solid red"),
                                          ]
                            },
                        ])


    return df
    
class Series(object):
    def __init__(self, series_id, match_id):
        # use
        self.url = f"https://www.espncricinfo.com/series/series-name-{series_id}/match-name-{match_id}/full-scorecard"
        self.page = requests.get(self.url)
        self.soup = BeautifulSoup(self.page.content, "html.parser")
        # self.match_id = str(self.soup.find_all(class_='ColumnistSmry')
        #                      [0]).split('.html')[0].split('/')[-1]
        # self.url_2 = f'https://www.espncricinfo.com/player/player-name-{self.player_id}'
        # self.page_2 = requests.get(self.url_2)
        # self.soup2 = BeautifulSoup(self.page_2.content, "html.parser")

        # self.temp_url = 'https://www.espncricinfo.com/player/player-name-{self.player_id}/tests-odi-t20-records'
        # self.temp_page = requests.get(self.temp_url)
        # self.temp_soup = BeautifulSoup(self.temp_page.content, "html.parser")

    def batting_df(self, bat_first):            
        if bat_first:
            df = pd.read_html(self.url)[0]   
        else:
            df = pd.read_html(self.url)[2]
        
        df = clean(df, 'bat')   
        return df

    def bowling_df(self, bowl_first):
        if bowl_first:
            df = pd.read_html(self.url)[1]   
        else:
            df = pd.read_html(self.url)[3]
        df = clean(df, 'bowl')   
        return df
    
    def test_bat(self, bat_first, inning):
        if bat_first and inning == 1:
            df = pd.read_html(self.url)[0]
        elif bat_first and inning == 2:
            df = pd.read_html(self.url)[4]
        elif not bat_first and inning == 1:
            df = pd.read_html(self.url)[2]
        elif not bat_first and inning == 2:
            df = pd.read_html(self.url)[6]
        df = clean(df, 'bat')   
        return df
            
            
            
    
    def test_bowl(self, bowl_first, inning):
        if bowl_first and inning == 1:
            df = pd.read_html(self.url)[1]
        if bowl_first and inning == 2:
            df = pd.read_html(self.url)[5]
        if not bowl_first and inning == 1:
            df = pd.read_html(self.url)[3]
        if not bowl_first and inning == 2:
            df = pd.read_html(self.url)[7]
        
        
        df = clean(df, 'bowl')   
        return df
            
        

    def get_general_df(self):
        df = pd.read_html(self.url)[6]
        df = clean(df, 'none')
        return df
    def get_standings(self):
        df = pd.read_html(self.url)[8]
        df = clean(df, 'none')
        return df
        
    def test_df(self):
        df = pd.read_html(self.url)[6]
        return df
        
        
        


In [18]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
Series(1276896, 1276916).get_general_df()



,0,1
0,"Lord's, London","Lord's, London"
1,Toss,"South Africa, elected to field first"
2,Series,South Africa tour of EnglandICC World Test Cha...
3,Season,2022
4,Player Of The Match,Kagiso Rabada
5,Series result,South Africa led the 3-match series 1-0
6,Match number,Test no. 2474
7,Match days,"17,18,19 August 2022 - day (5-day match)"
8,Umpires,Nitin MenonRichard Illingworth
9,TV Umpire,Chris Gaffaney


['/Users/aadrijupadya/CricCatapult/cricinfowrap', '/Library/Frameworks/Python.framework/Versions/3.8/lib/python38.zip', '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8', '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/lib-dynload', '', '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages']
